1:1 비율 down sampling


25에서 ACUM_RCPT_AMT가 1이상인 사람만 가지고 훈련

In [1]:
import pandas as pd
from lightgbm import Booster
import lightgbm as lgb
import copy
import time
import json
import joblib
import gc
import os.path as path
import numpy as np

In [2]:
train_path = '/home/workspace/user-workspace/slim_train.parquet'
test_path ='/home/workspace/user-workspace/slim_test.parquet'
encoder = '/home/workspace/user-workspace/cat_encoder.json'
decoder = '/home/workspace/user-workspace/inverse_cat_encoder.json'
data_dir = '/home/workspace/user-workspace/junheon/data/task150/'
model_dir = '/home/workspace/user-workspace/junheon/model/task150/'
result_dir = '/home/workspace/user-workspace/junheon/result/task150/'
submission_dir = '/home/workspace/user-workspace/prediction/'

In [3]:
bagging_size = 5
prefix = "ex27"

In [4]:
features = [
    "basic_feature",
    "day_weekday",
#     "pay_amt",
    "cp_count",
    "unique_count_wrt_cp",
    "average_wrt_cp",
    "prop_wrt_cp",
    "unique_divide_wrt_cp",
    "gods_count",
    "unique_count_wrt_gods",
    "average_wrt_gods",
    "prop_wrt_gods",
    "unique_divide_wrt_gods",
    "seq_count",
    "unique_count_wrt_seq",
    "last_transaction_diff",
    "ip_count_wrt_seq",
    "seq_count_wrt_ip",
    "newness",
#     "filter_CP_M_CLF_NM",
#     "filter_REMD_LMT_AMT",
#     "filter_ACUM_RCPT_AMT",
    "filter_CP_S_CLF_NM",
    "filter_SUB_IP_A",
    "is_KT",
    "bin_AC_PAY_AMT",
    "bin_MM_LMT_AMT",
    "bin_REMD_LMT_AMT",
    "bin_AGE",
    "bin_AUTHTI_CLF_FLG",
    "bin_ACUM_RCPT_AMT",
    "bin_CP_M_CLF_NM",
    "bin_CP_S_CLF_NM",
    "bin_NPAY_AMT_24M",
    "bin_AVG_AMT_6M",
    "bin_MAX_LMT_3M_RT",
    "bin_NPAY_CNT_24M",
    "bin_NPAY_AMT_60M"
]

In [5]:
categorical_features = [
    "NPAY_YN", "ARS_AUTHTI_YN", "PAY_MTHD_CD", "GNDR", "weekday",
    "high_risk_CP_S_CLF_NM", "high_risk_SUB_IP_A", "FOREI_YN", "SVC_CLF_NM"
]

In [6]:
categorical_features = categorical_features + ["COMMC_CLF", "AUTHTI_CLF_FLG", "SUB_IP_A", "CP_M_CLF_NM", "CP_S_CLF_NM"]

In [7]:
# remove_features = ["SUB_IP_A", "MM_LMT_AMT", "AC_PAY_AMT", "COMMC_CLF", "AUTHTI_CLF_FLG", "ACUM_RCPT_AMT",
#                   "CP_M_CLF_NM", "CP_S_CLF_NM", "NPAY_AMT_24M", "MAX_NPAY_CNT_24M", "TRD_CNT_6M", "REAL_TRD_CNT_6M", 
#                    "NIGHT_TRD_RT_6M", "AVG_AMT_6M", "MAX_LMT_3M_RT", "NPAY_CNT_24M", "NPAY_CNT_12MNTS", "NPAY_AMT_60M"]

In [8]:
remove_features = []

# merge data

### train

In [9]:
for seed in range(bagging_size):
    if path.isfile(f"{data_dir}{prefix}_train_{seed}.parquet"):
        continue
    df_list = []
    for name in features:
        feature_df = pd.read_parquet(f"{data_dir}{name}_{seed}.parquet").set_index("id")
        df_list.append(feature_df)
    df = pd.concat(df_list, axis=1)
    df = df.drop(columns=remove_features, axis=1)
    df.to_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")

### test

In [10]:
if not path.isfile(f"{data_dir}{prefix}_test.parquet"):
    df_list = []
    for name in features:
        feature_df = pd.read_parquet(f"{data_dir}{name}_test.parquet").set_index("id")
        df_list.append(feature_df)
    df = pd.concat(df_list, axis=1)
    df = df.drop(remove_features, axis=1)
    df.to_parquet(f"{data_dir}{prefix}_test.parquet")
else:
    df = pd.read_parquet(f"{data_dir}{prefix}_test.parquet")

In [11]:
if not path.isfile(f"{data_dir}{prefix}_test_1.parquet"):
    df[:1000000].to_parquet(f"{data_dir}{prefix}_test_1.parquet")
if not path.isfile(f"{data_dir}{prefix}_test_2.parquet"):
    df[1000000:2000000].to_parquet(f"{data_dir}{prefix}_test_2.parquet")
if not path.isfile(f"{data_dir}{prefix}_test_3.parquet"):
    df[2000000:3000000].to_parquet(f"{data_dir}{prefix}_test_3.parquet")
if not path.isfile(f"{data_dir}{prefix}_test_4.parquet"):
    df[3000000:].to_parquet(f"{data_dir}{prefix}_test_4.parquet")

# Modeling

In [12]:
model_params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "num_leaves": 255,
    "max_depth": 8,
    "min_child_samples": 200,
    "subsample": 0.9,
    "subsample_freq": 1,
    "colsample_bytree": 0.5,
    "min_child_weight": 0,
    "subsample_for_bin": 1000000,
    "min_split_gain": 0,
    "reg_lambda": 0,
    "verbose": 0,
    "seed": 777
}

train_params = {
    "num_boost_round": 5000,
    "early_stopping_rounds": 30
}

### train and valid

In [13]:
boosters = []
train_results = []

In [14]:
for seed in range(bagging_size):
    if path.isfile(f"{model_dir}{prefix}_model_{seed}.txt"):
        boosters.append(lgb.Booster(model_file=f"{model_dir}{prefix}_model_{seed}.txt"))
        continue
        
    start_time = time.time()
    
    train_df = pd.read_parquet(f"{data_dir}{prefix}_train_{seed}.parquet")
    valid_df = train_df[train_df['month']==10].drop(columns=['month'])
    train_df = train_df[train_df['month']!=10].drop(columns=['month'])
    train_dataset = lgb.Dataset(train_df.loc[:, train_df.columns != "target"], label=train_df["target"].values)
    valid_dataset = lgb.Dataset(valid_df.loc[:, valid_df.columns != "target"], label=valid_df["target"].values)
    eval_results = {}
    
    model: Booster = lgb.train(model_params,
                              train_dataset,
                              categorical_feature=categorical_features,
                              valid_sets=[train_dataset, valid_dataset],
                              valid_names=['train', 'valid'],
                              evals_result=eval_results,
                              **train_params)
        
    best_iteration = model.best_iteration
    
    params = copy.deepcopy(train_params)
    params["num_boost_round"] = best_iteration
    if "early_stopping_rounds" in params:
        del params["early_stopping_rounds"]
        
    train_df = pd.read_parquet(f"{data_dir}{prefix}_train_{seed}.parquet").drop(columns=['month'])
    train_dataset = lgb.Dataset(train_df.loc[:, train_df.columns != "target"], label=train_df["target"].values)
    model = lgb.train(model_params,
                     train_dataset,
                     categorical_feature=categorical_features,
                     **params)
    
    model.save_model(f"{model_dir}{prefix}_model_{seed}.txt")
    boosters.append(model)
    
    result = dict()
    result['train_auc'] = eval_results['train']['auc'][best_iteration-1]
    result['valid_auc'] = eval_results['valid']['auc'][best_iteration-1]
    result['best_iteration'] = best_iteration
    result['train_time'] = time.time() - start_time
    result['feature_importance'] = {name: int(score) for name, score in zip(model.feature_name(), model.feature_importance())}
    result['feature_importance'] = {key: value for key, value in sorted(result['feature_importance'].items(), key=lambda item: item[1], reverse=True)}
    train_results.append(result)

/home/workspace/user-workspace/lib/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ARS_AUTHTI_YN', 'AUTHTI_CLF_FLG', 'COMMC_CLF', 'CP_M_CLF_NM', 'CP_S_CLF_NM', 'FOREI_YN', 'GNDR', 'NPAY_YN', 'PAY_MTHD_CD', 'SUB_IP_A', 'SVC_CLF_NM', 'high_risk_CP_S_CLF_NM', 'high_risk_SUB_IP_A', 'weekday']
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015006 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/home/workspace/user-workspace/lib/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/workspace/user-workspace/lib/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's auc: 0.842937	valid's auc: 0.796628
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's auc: 0.865561	valid's auc: 0.838147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's auc: 0.874375	valid's auc: 0.843764
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's auc: 0.875988	valid's auc: 0.845593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's auc: 0.884094	valid's auc: 0.869324
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	train's auc: 0.88475	valid's auc: 0.871014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[7]	train's auc: 0.885172	valid's auc: 0.871241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[8]	

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[69]	train's auc: 0.905565	valid's auc: 0.894394
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	train's auc: 0.905753	valid's auc: 0.894548
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[71]	train's auc: 0.905765	valid's auc: 0.894511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	train's auc: 0.90595	valid's auc: 0.89457
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	train's auc: 0.906063	valid's auc: 0.894674
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	train's auc: 0.906235	valid's auc: 0.894891
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	train's auc: 0.906415	valid's auc: 0.894998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	train's auc: 0.906642	valid's auc: 0.895064
[LightGBM]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [15]:
if not path.isfile(f"{result_dir}{prefix}.result.json"):
    json.dump(train_results, open(f"{result_dir}{prefix}.result.json", 'w'), indent=2)

In [16]:
del train_df
del valid_df
gc.collect()

231

# Predict & Submission

In [17]:
# test_df = pd.read_parquet(f"{data_dir}{prefix}_test.parquet").drop(columns=['month'])

In [18]:
submission = pd.read_parquet(test_path)[['TRD_NO', 'ACUM_RCPT_AMT', 'CP_M_CLF_NM', 'REMD_LMT_AMT']]

In [19]:
predictions = []

In [20]:
predictions = []
for i, booster in enumerate(boosters):
    prediction = []
    print(f"prediction {i}")
    test_df = pd.read_parquet(f"{data_dir}{prefix}_test_1.parquet").drop(columns=['month'])
    prediction = prediction + booster.predict(test_df).tolist()
    test_df = pd.read_parquet(f"{data_dir}{prefix}_test_2.parquet").drop(columns=['month'])
    prediction = prediction + booster.predict(test_df).tolist()
    test_df = pd.read_parquet(f"{data_dir}{prefix}_test_3.parquet").drop(columns=['month'])
    prediction = prediction + booster.predict(test_df).tolist()
    test_df = pd.read_parquet(f"{data_dir}{prefix}_test_4.parquet").drop(columns=['month'])
    prediction = prediction + booster.predict(test_df).tolist()
    predictions.append(prediction)

prediction 0
prediction 1
prediction 2
prediction 3
prediction 4


In [21]:
for i, predict in enumerate(predictions):
    submission[f"target_{i}"] = predict

In [22]:
columns = [f"target_{i}" for i in range(len(predictions))]

In [23]:
submission['target'] = submission[columns].mean(1)
submission = submission.drop(columns=columns)

## EDA 기반 필터링

In [24]:
submission['target'] = np.where((submission['ACUM_RCPT_AMT']<1), 1, submission['target'])

In [25]:
submission['target'] = np.where((submission['REMD_LMT_AMT']>600000), 0, submission['target'])

In [26]:
submission['target'] = np.where((submission['CP_M_CLF_NM']==9), 0, submission['target'])

시간되면 이것도? CP_S_CLF_NM --> target_1인 train에 포함되냐 아니냐 여부 

## 제출 

In [27]:
submission.to_feather(f"{submission_dir}prediction_pre.feather")

In [28]:
submission = submission[['TRD_NO', 'target']]

In [29]:
submission.to_feather(f"{submission_dir}prediction.feather")

In [30]:
from nipa.taskSubmit import nipa_submit
import os

team_id = "1390"
task_no= "150"
prediction_path = f"{submission_dir}prediction.feather"
# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

# 제출 성공
nipa_submit(team_id=team_id,
            task_no=task_no,
            result=prediction_path
           )


is file:  True
20201114180705015701_TAbi.feather: 200


In [31]:
print(1)

1
